In [ ]:
# coding: utf-8


import os


def make_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print("Done!")
    else:
        print(f"{directory} exists!")


def generate_directory_tree(task_name):

    root_dir = os.path.join(
        '/home/yangqi/dl/cheminfo/spoc_20210904/spoc', 'benchmark', task_name)
    dir_list = ['features', 'log', 'output']

    for dir in dir_list:
        dir = os.path.join(root_dir, dir)
        make_dir(dir)


if __name__ == '__main__':

    task_list = ['tox21',]
    for task in task_list:
        print()
        print(f"Prepare to generate directory tree: {task}")
        print(f"{'-'*20}")
        generate_directory_tree(task)
        print(f"{'-'*20}")



Prepare to generate directory tree: tox21
--------------------
Done!
Done!
Done!
--------------------


In [ ]:
import pandas as pd
import numpy as np
import os
import deepchem as dc

## test splitter

split_mode = "SingletaskStratifiedSplitter"

data_file = "/home/yangqi/dl/cheminfo/spoc_20210904/spoc/data/SAMPL.csv"
descriptor_set_file = "/home/yangqi/dl/cheminfo/spoc_20210904/spoc/benchmark/SAMPL/features/single_features/all_descriptor_set--SAMPL.pkl.zip"

data_df = pd.read_csv(data_file, encoding='GBK')
desc_set_df = pd.read_pickle(descriptor_set_file)

X = np.array([list(desc_set_df["Mordred"][smi]) for smi in data_df["smiles"].values])
X = np.nan_to_num(X)
y = data_df["expt"].values
print(f"X.shape: {X.shape}")
print(f"y.shape: {y.shape}")

# data split
X_data = data_df["smiles"].values

# RandomSplitter
if split_mode == "RandomSplitter":
    splitter = dc.splits.RandomSplitter()
    dataset = dc.data.DiskDataset.from_numpy(X=X, y=y)

# ScaffoldSplitter
elif split_mode == "ScaffoldSplitter":
    splitter = dc.splits.ScaffoldSplitter()
    dataset = dc.data.NumpyDataset(X=X, y=y, ids=X_data)
    
# SingletaskStratifiedSplitter
elif split_mode == "SingletaskStratifiedSplitter":
    splitter = dc.splits.SingletaskStratifiedSplitter(task_number=0)
    y = np.expand_dims(y,axis=1)
    print(f"y.shape: {y.shape}")
    dataset = dc.data.DiskDataset.from_numpy(X=X, y=y)

train_dataset, test_dataset = splitter.train_test_split(dataset, frac_train=0.9, seed=1)

X_train = train_dataset.X
y_train = train_dataset.y
X_test = test_dataset.X
y_test = test_dataset.y


X.shape: (642, 1613)
y.shape: (642,)
y.shape: (642, 1)


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(576, 1613)
(576, 1)
(64, 1613)
(64, 1)


In [ ]:
print(test_dataset)

<DiskDataset X.shape: (64, 1613), y.shape: (64, 1), w.shape: (64, 1), ids: [171 609 476 ... 395 424 19], task_names: [0]>


In [ ]:
print(sorted(test_dataset.ids))

[2, 3, 13, 19, 57, 59, 68, 69, 74, 89, 90, 101, 111, 136, 150, 158, 167, 170, 171, 175, 198, 202, 203, 205, 213, 228, 242, 258, 269, 275, 291, 302, 312, 329, 346, 352, 360, 370, 388, 395, 400, 404, 419, 421, 424, 431, 448, 475, 476, 478, 501, 518, 544, 555, 561, 575, 587, 600, 604, 609, 614, 617, 628, 631]


In [ ]:
# randomsplitter

import deepchem as dc

X = np.array([
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['2','2','3,','4',],
    ['3','2','3,','4',],
    ['4','2','3,','4',],
])
y = [a+0.1 for a in list(range(10))]

dataset = dc.data.NumpyDataset(X,y)

splitter = dc.splits.RandomSplitter()
train, test = splitter.train_test_split(
    dataset, frac_train=0.8, seed=1)

train.y

array([2.1, 9.1, 6.1, 4.1, 0.1, 3.1, 1.1, 7.1])

In [ ]:
# SingletaskStratifiedSplitter

import deepchem as dc

X = np.array([
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['1','2','3,','4',],
    ['2','2','3,','4',],
    ['3','2','3,','4',],
    ['4','2','3,','4',],
])
y = np.array([a+0.1 for a in list(range(10))])
y = np.expand_dims(y,axis=1)

# dataset = dc.data.NumpyDataset(X=X, y=y, ids=list(range(10)))
dataset = dc.data.NumpyDataset(X=X, y=y)

splitter = dc.splits.SingletaskStratifiedSplitter()
train, test = splitter.train_test_split(
    dataset, frac_train=0.8, seed=1)

train

IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
import numpy as np
import pandas as pd

dataset = dc.data.DiskDataset.from_numpy(np.ones((10,10)), np.ones((10,1)))

splitter = dc.splits.SingletaskStratifiedSplitter(task_number=0)

train_dataset, test_dataset = splitter.train_test_split(dataset)
train_dataset.X

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
dataset = DiskDataset.from_numpy(np.ones((100,n_tasks)), np.ones((100,n_tasks)))

splitter = SingletaskStratifiedSplitter(task_number=5)

train_dataset, test_dataset = splitter.train_test_split(dataset)

NameError: name 'DiskDataset' is not defined

In [ ]:
dataset = dc.data.DiskDataset.from_numpy(np.ones((10,2)), np.ones((10,10)))

splitter = dc.splits.SingletaskStratifiedSplitter(task_number=5)

train_dataset, test_dataset = splitter.train_test_split(dataset)
train_dataset

<DiskDataset X.shape: (8, 2), y.shape: (8, 10), w.shape: (8, 1), ids: [8 3 5 9 0 6 1 7], task_names: [0 1 2 3 4 5 6 7 8 9]>

In [ ]:
y = np.array([a+0.1 for a in list(range(10))])
y.shape

(10,)

In [ ]:
y = np.expand_dims(y,axis=1)
y.shape

(10, 1)

In [ ]:
y[:,0]

array([0.1, 1.1, 2.1, 3.1, 4.1, 5.1, 6.1, 7.1, 8.1, 9.1])

In [ ]:
test

<NumpyDataset X.shape: (2, 4), y.shape: (2,), w.shape: (2,), ids: [8 5], task_names: [0]>

In [ ]:
dataset.y

array([0, 1, 2, 3, 4])

In [ ]:
dataset.X

array(['1', '2', '3', '4', '5'], dtype='<U1')

In [ ]:
import numpy as np
import pandas as pd

n_samples = 100

n_features = 10

n_tasks = 10

X = np.random.rand(n_samples, n_features)

y = np.random.rand(n_samples, n_tasks)

w = np.ones_like(y)

dataset = dc.data.np.ones((100,n_tasks)), np.ones((100,n_tasks))

splitter = dc.splits.SingletaskStratifiedSplitter(task_number=5)

train_dataset, test_dataset = splitter.train_test_split(dataset)

AttributeError: 'tuple' object has no attribute 'y'

In [ ]:
dataset = dc.data.np.ones((10,2)), np.ones((10,1))
dataset

(array([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]]),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]]))